# Laboratoire facultatif 
## Ce carnet web IPython ne fonctionne pas dans Google Colab mais il peut s'exécuter sur un poste de travail sur lequel est installé un fureteur piloté par Selenium.

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Moissonnage de données sur la Toile - formulaire web

## Les marées des sept prochains jours à Tadoussac 

<p>Vous planifiez une sortie en kayak de mer à Tadoussac qui est située sur le fleuve Saint-Laurent à l'embouchure du Saguenay. Vous allez moissonner les informations concernant les marées pour les sept prochains jours à compter du 21 septembre 2024. Pour cela, vous utiliserez un formulaire disponible sur le site de Pêches et Océans Canada. </p> 

<p>Pour simuler les interactions que fait un humain avec le site web, vous utiliserez la bibliothèque python <a href="https://selenium-python.readthedocs.io/" target='_blank'>Selenium</a>. L'extraction des sections pertinentes de la page web des résultats sera faite avec l'outil python <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target='_blank'>Beautiful Soup</a>. Enfin, des <a href="https://docs.python.org/fr/3/howto/regex.html" target='_blank'>expressions régulières<sup>1</sup></a> seront utilisées pour extraire les informations détaillées.</p>
<hr/>
<span style="font-size:80%"><sup>1</sup><b>Note - pratique: </b><a href="https://regex101.com/" target='_blank'>regex101.com</a> est un excellent site pour pratiquer et mettre au point des expressions régulières.</span>

## Importation des bibliothèques Python
<ul>
<ul>
  <li><b><i>re</i></b> (expressions régulières), <b><i>json</i></b> (traitement du format de données json: JavaScript Object Notation) et et <b><i>datetime</i></b> (traitement des dates et de l'heure) sont des bibliothèques Python standard, vous n'avez pas besoin de les installer, il suffit de les importer.</li>
    <li>Installation d'un <a href="https://selenium-python.readthedocs.io/installation.html#drivers" target='_blank'>pilote pour le contrôle de votre fureteur par Selenium</a></li>
  <ul>
      <li><a href="https://sites.google.com/a/chromium.org/chromedriver/downloads" target='_blank'>Pilotes</a> pour le fureteur Chrome</li> 
      <li>Par exemple sur Mac OS et le fureteur Chrome, brew install chromedriver</li>
  </ul>
  <li>Installation de <a href="https://selenium-python.readthedocs.io/" target='_blank'>Selenium</a></li>
  <ul>
    <li>Par exemple, sudo pip3 install selenium</li>
  </ul>
  <li>Installation de <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target='_blank'>Beautiful Soup</a></li>
  <ul>
    <li>Par exemple, sudo pip3 install bs4</li>
  </ul>
</ul>
</ul>

In [22]:
# Importation et installation de la bibliothèque Selenium
# Attention! insérez votre mot de passe (celui de votre ordinateur), sans guillemets
import sys
!echo votreMotDePasse | sudo -S {sys.executable} -m  pip install selenium

Password:WARNING: The directory '/Users/claudecoulombe/Library/Caches/pip' or its parent directory is not owned or is not writable by the current user. The cache has been disabled. Check the permissions and owner of that directory. If executing pip with sudo, you should use sudo's -H flag.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [23]:
# Importation et installation de la bibliothèque webdriver-manager
# Attention! insérez votre mot de passe (celui de votre ordinateur), sans guillemets
!echo votreMotDePasse | sudo -S {sys.executable} -m pip install webdriver-manager

Password:WARNING: The directory '/Users/claudecoulombe/Library/Caches/pip' or its parent directory is not owned or is not writable by the current user. The cache has been disabled. Check the permissions and owner of that directory. If executing pip with sudo, you should use sudo's -H flag.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [24]:
# Importation et installation de la bibliothèque chromedriver
# Par exemple, ici je télécharge le ChromeDriver pour Mac et l'installe dans le répertoire /usr/local/bin/
# Évidemment vous devez fournir votre mot de passe et le répertoire adéquat sur votre ordinateur
# Attention! insérez votre mot de passe (celui de votre ordinateur), sans guillemets
# Vous pourriez aussi devoir décompresser / unzip le fichier télécharger
!echo votreMotDePasse | sudo -S wget -O /usr/local/bin/chromedriver-mac-arm64.zip https://storage.googleapis.com/chrome-for-testing-public/128.0.6613.84/mac-arm64/chromedriver-mac-arm64.zip

Password:--2024-08-28 03:59:20--  https://storage.googleapis.com/chrome-for-testing-public/128.0.6613.84/mac-arm64/chromedriver-mac-arm64.zip
Résolution de storage.googleapis.com (storage.googleapis.com)… 172.217.13.187, 172.217.13.219
Connexion à storage.googleapis.com (storage.googleapis.com)|172.217.13.187|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 9628452 (9,2M) [application/zip]
Sauvegarde en : « /usr/local/bin/chromedriver-mac-arm64.zip »

/usr/local/bin/chro 100%[===================>]   9,18M  8,68MB/s    ds 1,1s    

2024-08-28 03:59:21 (8,68 MB/s) — « /usr/local/bin/chromedriver-mac-arm64.zip » sauvegardé [9628452/9628452]



In [25]:
# Installer la bibliothèque BeautifulSoup de manipulation du HTML  
# sudo pip3 install bs4
!echo votreMotDePasse | sudo -S {sys.executable} -m pip install bs4

Password:WARNING: The directory '/Users/claudecoulombe/Library/Caches/pip' or its parent directory is not owned or is not writable by the current user. The cache has been disabled. Check the permissions and owner of that directory. If executing pip with sudo, you should use sudo's -H flag.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [26]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from selenium import webdriver

import re
import pandas as pd

print("Bibliothèques Python importées")

Bibliothèques Python importées


## Analyse des points d'entrée du formulaire avec la fonction d'inspection de votre fureteur
<ul>
    <ul>
        <li>Ouvrir la page du <a href="https://www.marees.gc.ca/fra/station?sid=2985" target='_blank'>formulaire</a> sur le site de Pêches et Océans Canada;</li>
        <li>Reconstituer la séquence des interactions avec le formulaire (clics, menus et saisie clavier);</li>
        <li>Obtenir les identifiants des boutons, des menus, des champs de saisie;</li>
    </ul>
</ul>   

## Programmation d'un script d'interaction Selenium avec le formulaire des marées



In [44]:
# La nouvelle façon d'installer le pilote pour le fureteur Chrome
# mais pour cela, il faut réussir à installer la bibliothèque webdriver_manager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from warnings import filterwarnings
filterwarnings("ignore")

# Instancier un pilote web pour le fureteur Chrome
# service = Service('chemin vers chromedriver')
service = Service('/usr/local/bin/chromedriver-mac-arm64/chromedriver')

# pilote_chrome = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
pilote_chrome = webdriver.Chrome(service=service)

# Ouvrir la page web où se trouve le formulaire des marées
# Nous allons partir avec la station 00001
pilote_chrome.get("https://www.marees.gc.ca/fr/stations/00001")
# puis la changer pour la station "Tadoussac"
localite = "Tadoussac"

# Demander le formulaire pour la localité de Tadoussac avec l'outil de recherche / filtrage
# Pour trouver les identifiants HTML, il faut examiner la page web avec un outil d'inpection
# Pour le fureteur Chrome en faisant option-click sur la page Web et choisir « inspecter »
saisie_localite = pilote_chrome.find_element(By.ID,"station-select-list")
# saisie_localite.clear()
saisie_localite.send_keys(localite)

# Demander un délai d'exécution
import time 
time.sleep(2)

# Changer la date du composant calendrier, ici le 21 septembre 2024
nouvelle_date = "2024-09-21"
saisie_date = pilote_chrome.find_element(By.ID, "water-levels-date")
saisie_date.clear()
pilote_chrome.execute_script("arguments[0].value = '" + nouvelle_date + "';", saisie_date)

# Soumettre la requête pour obtenir l'horaire des marées à Tadoussac pour
# les sept prochains jours à compter du 21 septembre 2024
bouton_soumettre = pilote_chrome.find_element(By.ID, "water-levels-date-submit")
bouton_soumettre.click()

# Demander un délai d'exécution
time.sleep(5)

# Obtenir la page web des résultats
page_reponse = pilote_chrome.page_source

print("Le script Selenium devrait ouvrir un fureteur et afficher un message")
print("du genre: « Browser is being controlled by automated test software. »")
print("puis le script Selenium simule les interactions avec le formulaire.\n")
print("Script Selenium exécuté, formulaire soumis, page de résultats récupérée...")

Le script Selenium devrait ouvrir un fureteur et afficher un message
du genre: « Browser is being controlled by automated test software. »
puis le script Selenium simule les interactions avec le formulaire.

Script Selenium exécuté, formulaire soumis, page de résultats récupérée...


## Analyse de la page de résultats retournée avec `BeautifulSoup`

### Affichage formaté de la page de résultats en HTML 

In [45]:
# Demander un délai d'exécution
time.sleep(2)

# Analyser la page de réponse retournée avec BeautifulSoup
dom_page_resultats = BeautifulSoup(page_reponse,"html.parser") 

# Afficher la page web retournée
print(dom_page_resultats.prettify())

<html class="js js backgroundsize borderimage csstransitions fontface svg details progressbar meter mathml cors largeview wb-enable" dir="ltr" lang="fr" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <script src="https://www.marees.gc.ca/libraries/wet-boew/js/i18n/fr.min.js">
  </script>
  <script src="https://www.marees.gc.ca/libraries/wet-boew/js/deps/jquery.dataTables.min.js">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-W9G4T7DH4P&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script async="" src="https://www.google-analytics.com/analytics.js">
  </script>
  

## Extraction des sections pertinentes de la page de résultats avec `Beautiful Soup`

### Affichage des sections pertinentes contenant les données de marée pour sept jours

In [46]:
# Extraire de la page les sections pertinentes
# cela implique un examen du contenu de la page et
# une connaissance des possibilités de BeautifulSoup
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/

# Demander un délai d'exécution
time.sleep(2)

# Extraire les paramètres de la requête de recherche d'information
parametres_bruts = dom_page_resultats.find_all("table", {"class":"day-tables clearfix table-bordered responsive-enabled table table-hover table-striped"})
print("Sections pertinentes en HTML:\n")
print("Nombre de jours:",len(parametres_bruts),"\n")
print(parametres_bruts)
# parametres_bruts = parametres_bruts.find("div", {"class":"wb-eqht wb-init wb-eqht-inited"})


Sections pertinentes en HTML:

Nombre de jours: 7 

[<table aria-label="Times and Heights for High and Low Tides for 2024-09-21" class="day-tables clearfix table-bordered responsive-enabled table table-hover table-striped" data-striping="1" id="day-table-2024-09-21" style="font-size: 0.8em">
<caption>2024-09-21 (sam)</caption>
<thead>
<tr>
<th>Heure HAA</th>
<th>Niveaux (m)</th>
<th>Niveaux (pi)</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td>06:29</td>
<td>4.7</td>
<td>15.3</td>
</tr>
<tr class="even">
<td>12:31</td>
<td>0.1</td>
<td>0.3</td>
</tr>
<tr class="odd">
<td>18:47</td>
<td>5</td>
<td>16.5</td>
</tr>
</tbody>
</table>, <table aria-label="Times and Heights for High and Low Tides for 2024-09-22" class="day-tables clearfix table-bordered responsive-enabled table table-hover table-striped" data-striping="1" id="day-table-2024-09-22" style="font-size: 0.8em">
<caption>2024-09-22 (dim)</caption>
<thead>
<tr>
<th>Heure HAA</th>
<th>Niveaux (m)</th>
<th>Niveaux (pi)</th>
</tr>
</th

### Affichage du texte brut sans les balises HTML

In [47]:
print("Sections pertinentes en texte brut sans les balises HTML:\n")
for parametre_brut in parametres_bruts:
    texte_affichage = parametre_brut.getText().strip()
    print(texte_affichage)

Sections pertinentes en texte brut sans les balises HTML:

2024-09-21 (sam)


Heure HAA
Niveaux (m)
Niveaux (pi)




06:29
4.7
15.3


12:31
0.1
0.3


18:47
5
16.5
2024-09-22 (dim)


Heure HAA
Niveaux (m)
Niveaux (pi)




01:09
0
-0.1


07:14
4.3
14.1


13:11
0.4
1.3


19:33
4.8
15.7
2024-09-23 (lun)


Heure HAA
Niveaux (m)
Niveaux (pi)




02:01
0.3
1.1


08:04
3.8
12.6


13:56
0.8
2.5


20:24
4.4
14.6
2024-09-24 (mar)


Heure HAA
Niveaux (m)
Niveaux (pi)




03:01
0.8
2.6


09:00
3.4
11.2


14:48
1.2
3.8


21:26
4.1
13.4
2024-09-25 (mer)


Heure HAA
Niveaux (m)
Niveaux (pi)




04:19
1.2
3.8


10:11
3.1
10


16:01
1.5
4.9


22:42
3.8
12.5
2024-09-26 (Jeu)


Heure HAA
Niveaux (m)
Niveaux (pi)




05:53
1.3
4.3


11:41
2.9
9.6


17:39
1.6
5.3
2024-09-27 (ven)


Heure HAA
Niveaux (m)
Niveaux (pi)




00:11
3.7
12.3


07:16
1.3
4.1


13:09
3
9.9


19:07
1.5
4.9


## Affichage avec rendu des tableaux HTML de marées

In [48]:
import re
from IPython.display import display, HTML

# Extraction des attributs au moyen d'une expression régulière
# la mise au point de l'expression régulière se fait avec https://regex101.com/

FORME_1 = r"(?:.*\n)<caption>(\d\d\d\d-\d\d-\d\d\s*\(\w*\))\<\/caption>"

for parametre_brut in parametres_bruts:
    titre = re.match(re.compile(FORME_1),str(parametre_brut)).group(1)
    substitution = re.sub(FORME_1, '<table>', str(parametre_brut))
    display(HTML(titre+"\n"+substitution))

Heure HAA,Niveaux (m),Niveaux (pi)
06:29,4.7,15.3
12:31,0.1,0.3
18:47,5,16.5


Heure HAA,Niveaux (m),Niveaux (pi)
01:09,0,-0.1
07:14,4.3,14.1
13:11,0.4,1.3
19:33,4.8,15.7


Heure HAA,Niveaux (m),Niveaux (pi)
02:01,0.3,1.1
08:04,3.8,12.6
13:56,0.8,2.5
20:24,4.4,14.6


Heure HAA,Niveaux (m),Niveaux (pi)
03:01,0.8,2.6
09:00,3.4,11.2
14:48,1.2,3.8
21:26,4.1,13.4


Heure HAA,Niveaux (m),Niveaux (pi)
04:19,1.2,3.8
10:11,3.1,10
16:01,1.5,4.9
22:42,3.8,12.5


Heure HAA,Niveaux (m),Niveaux (pi)
05:53,1.3,4.3
11:41,2.9,9.6
17:39,1.6,5.3


Heure HAA,Niveaux (m),Niveaux (pi)
00:11,3.7,12.3
07:16,1.3,4.1
13:09,3,9.9
19:07,1.5,4.9


## Extraction des données de marées au moyen d'expressions régulières

In [49]:
def extraire_numero_station(texte_entree):
    FORME_1 = r'\s+\((\d*)\)\s+|\s+Pêches\s+et\s+Océans Canada<\/title>'
    return re.search(FORME_1, str(texte_entree)).group(1)

numero_station = extraire_numero_station(dom_page_resultats)
print("Numéro de station:",numero_station)

Numéro de station: 03425


In [50]:
def extraire_fuseau_horaire(texte_entree):
    FORME_2 = r'<th>Heure\s*(\w*)<\/th>'
    return re.search(FORME_2, str(texte_entree)).group(1)

fuseau_horaire = extraire_fuseau_horaire(parametres_bruts[0])
print("Fuseau horaire:",fuseau_horaire)

Fuseau horaire: HAA


In [51]:
from datetime import datetime

def extraire_date(texte_entree):
    FORME_3 = r"(\d\d\d\d)-(\d\d)-(\d\d)"
    resultats_extraction = re.search(re.compile(FORME_3),str(texte_entree))
    # Réécriture de la date en français
    date_str = resultats_extraction.group(3) + "-" + resultats_extraction.group(2) + "-" + resultats_extraction.group(1) 
    return date_str

extraire_date(parametres_bruts[0])

'21-09-2024'

In [52]:
def extraire_donnees_maree(texte_entree):
    FORME_4 = r"<td>(\d*:\d*)<\/td>\n<td>(\d*.\d*)<\/td>"
    resultats_extraction = re.findall(re.compile(FORME_4),str(texte_entree))
    return resultats_extraction

extraire_donnees_maree(parametres_bruts[0])

[('06:29', '4.7'), ('12:31', '0.1'), ('18:47', '5')]

## Sauvegarde des données dans le fichier `donnees_marees_formulaire_web.csv`

In [40]:
# Sauvegarder les données dans un fichier .csv
chemin_fichier_sortie = './'
nom_fichier_sortie = "donnees_marees_formulaire_web.csv"
with open(chemin_fichier_sortie+nom_fichier_sortie,'w') as fichier_sortie:
    # Écriture de l'entête du fichier listant les différents attributs
    fichier_sortie.write('localite\tstation\tfuseau_horaire\tdate\theure\thauteur_m\n')
    for parametre_brut in parametres_bruts:
        titre = re.match(re.compile(FORME_1),str(parametre_brut)).group(1)
        date = extraire_date(parametre_brut)
        donnees_marees_liste = extraire_donnees_maree(parametre_brut)
        for donnees_marees in donnees_marees_liste:
            # Écriture des données sur la marée dans le fichier
            fichier_sortie.write(localite+'\t'+numero_station+'\t'+fuseau_horaire+'\t'+date+'\t'
                                 +donnees_marees[0]+'\t'+donnees_marees[1]+'\n')
        
print("Données de marées sauvegardées dans le fichier "+ nom_fichier_sortie)


Données de marées sauvegardées dans le fichier donnees_marees_formulaire_web.csv


## Test de lecture du fichier de données

Vérification que le fichier .csv est correct.

In [41]:
import pandas as pd

nom_fichier_sortie = "donnees_marees_formulaire_web.csv"


donnees_marees_df = pd.read_csv(chemin_fichier_sortie+nom_fichier_sortie,delimiter='\t')
donnees_marees_df

,localite,station,fuseau_horaire,date,heure,hauteur_m
0,Tadoussac,3425,HAA,21-09-2024,06:29,4.7
1,Tadoussac,3425,HAA,21-09-2024,12:31,0.1
2,Tadoussac,3425,HAA,21-09-2024,18:47,5.0
3,Tadoussac,3425,HAA,22-09-2024,01:09,0.0
4,Tadoussac,3425,HAA,22-09-2024,07:14,4.3
5,Tadoussac,3425,HAA,22-09-2024,13:11,0.4
6,Tadoussac,3425,HAA,22-09-2024,19:33,4.8
7,Tadoussac,3425,HAA,23-09-2024,02:01,0.3
8,Tadoussac,3425,HAA,23-09-2024,08:04,3.8
9,Tadoussac,3425,HAA,23-09-2024,13:56,0.8
